<a href="https://colab.research.google.com/github/arebs23/Spooky-Author-identification/blob/main/Spooky_Author.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a project on spooky author identification with traditional ML and deep learning approach. The data is available on Kaggle.  <br><br>
Kaggle Competition: <br>
https://www.kaggle.com/c/spooky-author-identification

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import xgboost as xgb
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition,model_selection,metrics,pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPool1D, Conv1D, MaxPool1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence,text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
stop_words = stopwords.words('english')



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
from zipfile import ZipFile
file_name = 'spooky-author-identification.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
from zipfile import ZipFile
file_name = 'test.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
from zipfile import ZipFile
file_name = 'train.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [ ]:
def multiclass_logloss(actual,predicted, eps = 1e-15):
  if len(actual.shape) == 1:
    actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
    for i, val in enumerate(actual):
      actual2[i, val] = 1
    actual = actual2

  
  clip = np.clip(predicted, eps, 1 - eps)
  rows = actual.shape[0]
  vsota  = np.sum(actual * np.log(clip))
  return -1.0/rows * vsota

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(train.text.values,y,stratify = y,random_state = 42,test_size = 0.1, shuffle  = True)

In [ ]:
print(x_train.shape)
print(x_valid.shape)

(17621,)
(1958,)


In [ ]:
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',
                      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3),use_idf=1, 
                      smooth_idf=1,sublinear_tf=1,stop_words = 'english')

tfv.fit(list(x_train)+list(x_valid))
x_train_tfv = tfv.transform(x_train)
x_valid_tfv = tfv.transform(x_valid)

In [ ]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv, y_train)
predictions = clf.predict_proba(x_valid_tfv)

print("logloss: %0.3f" % multiclass_logloss(y_valid,predictions))

logloss: 0.572


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf = MultinomialNB()
clf.fit(x_train_tfv,y_train)
predictions = clf.predict_proba(x_valid_tfv)

print("logloss: %0.3f" % multiclass_logloss(y_valid,predictions))

logloss: 0.578


In [ ]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(x_train_tfv)
x_train_svd = svd.transform(x_train_tfv)
x_valid_svd = svd.transform(x_valid_tfv)


scl = preprocessing.StandardScaler()
scl.fit(x_train_svd)
x_train_svd_scl = scl.transform(x_train_svd)
x_valid_svd_scl = scl.transform(x_valid_svd)

In [ ]:
clf = SVC(C=1.0,probability=True)
clf.fit(x_train_svd_scl,y_train)
prediction = clf.predict_proba(x_valid_svd_scl)

print("logloss: %0.3f" % multiclass_logloss(y_valid,predictions))

logloss: 0.578


In [ ]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(x_train_svd,y_train)
predictions = clf.predict_proba(x_valid_svd)


print("logloss: %0.3f" % multiclass_logloss(y_valid,predictions))

logloss: 0.816


In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False,needs_proba=True)

In [ ]:
svd = TruncatedSVD()

scl =  preprocessing.StandardScaler()

lr_model = LogisticRegression()

clf = pipeline.Pipeline([('svd', svd),('scl',scl),('lr',lr_model)])

In [ ]:
param_grid = {'svd__n_components':[120,180],
              'lr__C':[0.1,1.0,10],'lr__penalty':['l1','l2']}

In [ ]:
model = GridSearchCV(estimator = clf,param_grid=param_grid, 
                     scoring=mll_scorer,verbose=10,
                     n_jobs = -1,iid = True,refit = True,cv=2)

model.fit(x_train_tfv,y_train)

print('best score: %0.3f' % model.best_score_)
print('Best parameters set:')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
  print("\t%s: %r" %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   33.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   33.7s finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


best score: -0.739
Best parameters set:
	lr__C: 0.1
	lr__penalty: 'l2'
	svd__n_components: 180


In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2021-05-12 16:37:36--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-05-12 16:37:37--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-05-12 16:37:37--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
from zipfile import ZipFile
file_name = 'glove.840B.300d.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


### Word2Vec

In [ ]:
def load_glove_index():
  EMBEDDING_FILE = '/content/glove.840B.300d.txt'
  def get_coefs(word, *arr):return word, np.asarray(arr, dtype = 'float32')[100:]
  embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
  return embeddings_index

In [ ]:
embeddings_index = load_glove_index()
print('Found %s word vectors' % len(embeddings_index))

Found 2196016 word vectors


In [ ]:
def sent2vec(s):
  words  = str(s).lower()
  words = word_tokenize(words)
  words = [w for w in words if not w in stop_words]
  words = [w for w in words if w.isalpha()]
  M  = []
  for w in words:
    try:
      M.append(embeddings_index[w])
    except:
      continue
  
  M = np.array(M)
  v = M.sum(axis = 0)
  if type(v) != np.ndarray:
    return np.zeros(300)
  return v / np.sqrt((v ** 2).sum())


In [ ]:
x_train_glove = [sent2vec(x) for x in tqdm(x_train)]
x_valid_glove = [sent2vec(x) for x in tqdm(x_valid)]

In [ ]:
x_train_glove = np.array(x_train_glove)

In [ ]:
x_valid_glove = np.array(x_valid_glove)

In [ ]:
x_train_glove.shape

(17621,)

In [ ]:
x_train_tfv.shape

(17621, 15102)

In [ ]:
log_clf = LogisticRegression(C=1.0)

In [ ]:
xgb_clf = xgb.XGBClassifier(nthread=10)

In [ ]:
xgb_clf.fit(x_train_glove, y_train) 

IndexError: ignored

In [ ]:
log_clf.fit(x_train_glove, y_train)

ValueError: ignored

In [ ]:
predictions = clf.predict_proba(x_valid_glove)
print("logloss: %0.3f" % multiclass_logloss(y_valid,predictions))

In [ ]:
x_train_glove.dtype

dtype('O')

In [ ]:
scl = preprocessing.StandardScaler()
x_train_glove_scl = scl.fit_transform(x_train_glove)
x_valid_glove_scl = scl.fit_transform(x_valid_glove)

# Let Try a Neural Network


### Preprocessing steps

In [ ]:
token = text.Tokenizer()
max_len = 70

token.fit_on_texts(list(x_train)+list(x_valid))
x_train_seq = token.texts_to_sequences(x_train)
x_valid_seq = token.texts_to_sequences(x_valid)


x_train_pad = sequence.pad_sequences(x_train_seq, maxlen=max_len)
x_valid_pad = sequence.pad_sequences(x_valid_seq, maxlen=max_len)

In [ ]:
word_index = token.word_index

In [ ]:
embedding_matrix = np.zeros((len(word_index)+1,200))
for word, i in tqdm(word_index.items()):
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


100%|██████████| 25943/25943 [00:00<00:00, 322814.89it/s]


In [ ]:
y_train_enc = np_utils.to_categorical(y_train)
y_valid_enc =  np_utils.to_categorical(y_valid)

#### Model

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index)+1,200,
                    weights = [embedding_matrix],input_length = max_len,trainable = False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer ='adam',metrics = ['acc'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 70, 200)           5188800   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 70, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              103424    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)             

In [ ]:
model.fit(x_train_pad,y_train_enc, batch_size = 512, epochs = 100,verbose = 1,validation_data = (x_valid_pad,y_valid_enc))

Epoch 1/100
35/35 [==============================] - 18s 359ms/step - loss: 1.0898 - acc: 0.3952 - val_loss: 0.9334 - val_acc: 0.5807
Epoch 2/100
35/35 [==============================] - 12s 356ms/step - loss: 0.9422 - acc: 0.5564 - val_loss: 0.7859 - val_acc: 0.6742
Epoch 3/100
35/35 [==============================] - 12s 353ms/step - loss: 0.8604 - acc: 0.6155 - val_loss: 0.7623 - val_acc: 0.6639
Epoch 4/100
35/35 [==============================] - 12s 342ms/step - loss: 0.8268 - acc: 0.6347 - val_loss: 0.7281 - val_acc: 0.7043
Epoch 5/100
35/35 [==============================] - 12s 352ms/step - loss: 0.7943 - acc: 0.6499 - val_loss: 0.7128 - val_acc: 0.7099
Epoch 6/100
35/35 [==============================] - 12s 342ms/step - loss: 0.7855 - acc: 0.6649 - val_loss: 0.6950 - val_acc: 0.7155
Epoch 7/100
35/35 [==============================] - 12s 348ms/step - loss: 0.7716 - acc: 0.6650 - val_loss: 0.6925 - val_acc: 0.7084
Epoch 8/100
35/35 [==============================] - 12s 354ms

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
loss = history.history['loss']
epochs = range(1,len(acc) + 1)
plt.plot(epochs,acc,'bo',label = 'Training acc')
plt.plot(epochs,acc,'b',label = 'validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'bo',label = 'Training loss')
plt.plot(epochs,val_loss,'b',label = 'validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#save the model
model.save_weights('model.h5')

In [ ]:
#load model
model = load_model('model.h5')
#evaluate on test dataset
_,acc = model.evaluate()